# 🚀 HVAC AI — Production-Ready YOLO11 Inference Server
**Optimized Turn-Key Backend/Inference Notebook**

---

## 📋 Overview
Production-ready YOLO11 inference deployment with:
- ✅ Comprehensive GPU & dependency validation
- ✅ Optimized configuration management
- ✅ Error handling & monitoring
- ✅ Testing & benchmarking
- ✅ Security best practices
- ✅ Turn-key deployment

## 🎯 Prerequisites
1. **GPU Runtime**: T4 or better (Runtime → Change runtime type → GPU)
2. **Trained Model**: YOLO11 `.pt` file in Google Drive
3. **Ngrok Token**: Free token from [ngrok.com](https://ngrok.com/)
4. **Test Image**: Sample HVAC blueprint


In [ ]:
import sys
import os

print("="*70)
print("🔧 Environment Setup & Validation")
print("="*70)

# Clone repository
print("\n📦 Cloning repository...")
!git clone https://github.com/elliotttmiller/hvac-ai.git 2>/dev/null || echo "Repository exists"
%cd hvac-ai

# Install dependencies
print("\n📚 Installing dependencies (2-3 minutes)...")
!pip install -q ultralytics>=8.0.0 fastapi>=0.115.0 uvicorn[standard]>=0.34.0
!pip install -q python-multipart>=0.0.9 pyngrok>=7.0.0 python-dotenv>=1.0.0
!pip install -q Pillow>=10.0.0 numpy>=1.24.0 tqdm>=4.65.0

# Validate environment
print("\n🔍 System Validation")
print("="*70)

import torch
print(f"🐍 Python: {sys.version.split()[0]}")
print(f"🔥 PyTorch: {torch.__version__}")

if torch.cuda.is_available():
    print(f"\n✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"   CUDA: {torch.version.cuda}")
    # Test GPU
    test_tensor = torch.rand(1000, 1000).cuda()
    _ = torch.matmul(test_tensor, test_tensor)
    print(f"   Test: ✅ PASSED")
else:
    print("\n⚠️  WARNING: No GPU! Set Runtime > GPU. Inference will be SLOW.")

print("\n✅ Environment Ready!")
print('=' * 70)

In [ ]:
# Mount Google Drive for model access
from google.colab import drive
drive.mount('/content/drive')
print("✅ Drive mounted at: /content/drive/MyDrive")

In [ ]:
import os
from pathlib import Path

print("⚙️  Configuration")
print("="*70)

# --- UPDATE THESE VALUES ---
MODEL_PATH = "/content/drive/MyDrive/hvac_detection_project/runs/segment/hvac_yolo11_segmentation_sota/weights/best.pt"
NGROK_AUTHTOKEN = "YOUR_NGROK_TOKEN_HERE"  # Get from ngrok.com

# Server settings
PORT = 8000
DEFAULT_CONF_THRESHOLD = 0.50
DEFAULT_IOU_THRESHOLD = 0.45
MAX_IMAGE_SIZE = 1024

# Validation
errors = []
if not MODEL_PATH or not os.path.exists(MODEL_PATH):
    errors.append("❌ MODEL_PATH invalid or not found")
else:
    print(f"✅ Model: {MODEL_PATH}")
    print(f"   Size: {os.path.getsize(MODEL_PATH) / 1e6:.1f} MB")

if not NGROK_AUTHTOKEN or NGROK_AUTHTOKEN == "YOUR_NGROK_TOKEN_HERE":
    print("⚠️  Ngrok token not set (optional, for public URL)")
else:
    print(f"✅ Ngrok: {'*' * 20}{NGROK_AUTHTOKEN[-8:]}")

print(f"\n🎯 Inference: conf={DEFAULT_CONF_THRESHOLD}, iou={DEFAULT_IOU_THRESHOLD}, size={MAX_IMAGE_SIZE}")

# Write .env
with open('.env', 'w') as f:
    f.write(f"MODEL_PATH={MODEL_PATH}\nNGROK_AUTHTOKEN={NGROK_AUTHTOKEN}\nPORT={PORT}\n")

if errors:
    print("\n❌ Errors:", "\n".join(errors))
else:
    print("\n✅ Configuration valid")
print("="*70)

In [ ]:
import torch
import numpy as np
import time
from ultralytics import YOLO

print("🤖 Model Loading & Validation")
print("="*70)

print(f"\n📥 Loading model (10-30s)...")
start = time.time()
model = YOLO(MODEL_PATH)
print(f"✅ Loaded in {time.time() - start:.2f}s")

print(f"\n📊 Model Info:")
print(f"   Device: {model.device}")
print(f"   Classes: {len(model.names)}")
for idx, name in model.names.items():
    print(f"   [{idx}] {name}")

if torch.cuda.is_available():
    model.to('cuda')
    print(f"\n🚀 Model on GPU")

# Warm-up
print(f"\n🔥 Warm-up inference...")
dummy = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)
start = time.time()
_ = model.predict(dummy, verbose=False, conf=0.25)
first_time = time.time() - start
start = time.time()
_ = model.predict(dummy, verbose=False, conf=0.25)
second_time = time.time() - start

print(f"   First: {first_time*1000:.1f}ms")
print(f"   Subsequent: {second_time*1000:.1f}ms (~{1.0/second_time:.0f} FPS)")

if torch.cuda.is_available():
    print(f"\n💾 GPU Memory: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")

print("\n✅ Model ready!")
print("="*70)

In [ ]:
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt
import time

print("🧪 Test Inference")
print("="*70)

print("\n📤 Upload test image...")
uploaded = files.upload()

if uploaded:
    img_path = list(uploaded.keys())[0]
    img = Image.open(img_path).convert('RGB')
    img_array = np.array(img)
    
    print(f"\n📊 Image: {img.size[0]}x{img.size[1]}")
    
    print(f"\n🔄 Running inference...")
    start = time.time()
    results = model.predict(img_array, conf=DEFAULT_CONF_THRESHOLD, 
                           iou=DEFAULT_IOU_THRESHOLD, imgsz=MAX_IMAGE_SIZE, verbose=False)
    inf_time = (time.time() - start) * 1000
    
    result = results[0]
    boxes = result.boxes
    
    print(f"\n✅ Complete: {inf_time:.1f}ms ({1000.0/inf_time:.1f} FPS)")
    print(f"   Detections: {len(boxes)}")
    
    if len(boxes) > 0:
        class_counts = {}
        for box in boxes:
            cls_id = int(box.cls[0])
            name = model.names[cls_id]
            class_counts[name] = class_counts.get(name, 0) + 1
        print(f"\n📊 By Class:")
        for name, count in sorted(class_counts.items()):
            print(f"   {name}: {count}")
    
    # Visualize
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    ax1.imshow(img_array)
    ax1.set_title('Original')
    ax1.axis('off')
    ax2.imshow(result.plot())
    ax2.set_title(f'{len(boxes)} detections ({inf_time:.0f}ms)')
    ax2.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("❌ No image uploaded")

In [ ]:
from pyngrok import ngrok
from dotenv import load_dotenv
import os

print("🚀 Deploying API Server")
print("="*70)

load_dotenv()

# Validate configuration
if not os.path.exists(MODEL_PATH):
    print("\n❌ ERROR: MODEL_PATH not found. Check configuration.")
    raise FileNotFoundError(f"Model not found: {MODEL_PATH}")

print(f"✅ Model found: {MODEL_PATH}")

# Setup ngrok tunnel
if NGROK_AUTHTOKEN and NGROK_AUTHTOKEN != "YOUR_NGROK_TOKEN_HERE":
    print("\n🌐 Setting up ngrok tunnel...")
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    public_url = ngrok.connect(PORT)
    print(f"\n✅ API LIVE!")
    print(f"   Public URL: {public_url.public_url}")
    print(f"   API Docs: {public_url.public_url}/docs")
    print(f"   Health: {public_url.public_url}/health")
else:
    print("\n⚠️  No ngrok token - server will be local only")
    print(f"   Local URL: http://localhost:{PORT}")

print("\n" + "="*70)
print("🎬 Starting server (Press STOP button to shutdown)...")
print("="*70 + "\n")

# Validate python-services directory exists
if not os.path.exists('python-services'):
    print("❌ ERROR: python-services directory not found")
    print(f"   Current directory: {os.getcwd()}")
    print("   Please ensure you're in the hvac-ai repository root")
    raise FileNotFoundError("python-services directory not found")

%cd python-services
# Use PORT variable via Python string formatting
import subprocess
subprocess.run(["uvicorn", "hvac_analysis_service:app", "--host", "0.0.0.0", "--port", str(PORT), "--reload"])